In [1]:
import os, sys
sys.path.insert(1, os.path.join(sys.path[0], '../..'))
from sklearn.datasets import load_iris
import scipy.cluster.hierarchy as sch
import idendrogram
#plot in Altair
from idendrogram.targets.altair import to_altair
import altair as alt

In [3]:
alt.renderers.set_embed_options(actions=False)
data = load_iris(as_frame=True)
linkage_matrix = sch.linkage(data['data'], method='single', metric='euclidean')
threshold = 0.8
flat_clusters = sch.fcluster(linkage_matrix, t=threshold, criterion='distance')
cl_data = idendrogram.ClusteringData(linkage_matrix = linkage_matrix, cluster_assignments = flat_clusters)

# Customizing other attributes

## Changing default values

Default link, node and axis label values can be changed by providing customized data classes to idendrogram constructor. For a full list of properties of each data class, see [Data Containers](../data_containers).

In [20]:
from idendrogram.containers import AxisLabel, ClusterLink, ClusterNode
from dataclasses import dataclass, field

@dataclass
class MyAxisLabel(AxisLabel):
    labelAngle: int = -180

@dataclass
class MyNode(ClusterNode):
    radius: float = 5
    _default_leaf_radius: float = 10    

@dataclass
class Mylink(ClusterLink):
    strokewidth: float = 0.5
    strokedash: list = field(default_factory=lambda: [6, 2])
    
idd = idendrogram.idendrogram(
    axis_label_factory=lambda x:  MyAxisLabel(**x),
    node_factory= lambda x: MyNode(**x),
    link_factory= lambda x: Mylink(**x))

idd.set_cluster_info(cl_data)
dendrogram = idd.create_dendrogram()
to_altair(dendrogram=dendrogram, height=200, width=629)

alt.LayerChart(...)

## Postprocessing

Any attribute of a node or a link can also be adjusted after the dendrogram object has been created. Here's an example where radii of nodes are adjusted based on number of observations they represent.

In [25]:
idd = idendrogram.idendrogram()
idd.set_cluster_info(cl_data)
dendrogram = idd.create_dendrogram()

for n in dendrogram.nodes:
    obs_count = int(idendrogram.callbacks.counts(cl_data, n.id))
    n.radius = obs_count / 10


to_altair(dendrogram=dendrogram, height=200, width=629)

alt.LayerChart(...)